# I will read Takashi-PJT's convection models

In [1]:
using Interpolations

In [2]:
dir="/Users/nobuaki/Documents/MantleConvectionTakashi/data2025"

"/Users/nobuaki/Documents/MantleConvectionTakashi/data2025"

In [3]:
function myListDir(dir;pattern=nothing)
    list=readdir(dir)
    newlist=[]
    if pattern != nothing
        for element in list
            if match(pattern,element) != nothing
                newlist = push!(newlist,dir*"/"*element)
            end
        end
    else 
        newlist=list
    end
    
    return newlist
end


myListDir (generic function with 1 method)

In [4]:
compositions=myListDir(dir; pattern=r"test_c\d")
temperatures=myListDir(dir; pattern=r"test_t\d")

127-element Vector{Any}:
 "/Users/nobuaki/Documents/MantleConvectionTakashi/data2025/test_t00000"
 "/Users/nobuaki/Documents/MantleConvectionTakashi/data2025/test_t00001"
 "/Users/nobuaki/Documents/MantleConvectionTakashi/data2025/test_t00002"
 "/Users/nobuaki/Documents/MantleConvectionTakashi/data2025/test_t00003"
 "/Users/nobuaki/Documents/MantleConvectionTakashi/data2025/test_t00004"
 "/Users/nobuaki/Documents/MantleConvectionTakashi/data2025/test_t00005"
 "/Users/nobuaki/Documents/MantleConvectionTakashi/data2025/test_t00006"
 "/Users/nobuaki/Documents/MantleConvectionTakashi/data2025/test_t00007"
 "/Users/nobuaki/Documents/MantleConvectionTakashi/data2025/test_t00008"
 "/Users/nobuaki/Documents/MantleConvectionTakashi/data2025/test_t00009"
 ⋮
 "/Users/nobuaki/Documents/MantleConvectionTakashi/data2025/test_t00118"
 "/Users/nobuaki/Documents/MantleConvectionTakashi/data2025/test_t00119"
 "/Users/nobuaki/Documents/MantleConvectionTakashi/data2025/test_t00120"
 "/Users/nobuaki/Docume

In [5]:

fieldcen=("t    ","vp   ","eta  ","c    ","f    ","fc   ","df   ","vm   ","ed   ","str  ", "rho  ","k    ","vd   ","ah   ","H    ","nrh  ","ph   ","ly   ","bs   ","hz   ", "TTG  ","bascr","TTGcr","cc   ","air  ","prm  ","hpe  ","imp  ","wtr  ","crb  ","ntg  ","mtl  ","FeO  ","age  ","disl ","gbs  ","Peier","plast","gs   ","defmd","Cp   ","expan","Pdyn ","tpc  ","trho ","Wsol ","Bvel ","Pvel ","Svel ","phs11","phs21","phs31","phs41","phs51","phs12","phs22","phs32","phs42","phs52")
fieldname=("Temperature         ","Pressure            ","Viscosity           ","Composition         ","Melt_fraction       ","Melt_composition    ","dmelt/dt            ","Melt_velocity       ","Strain_rate         ","Stress              ","Density             ","Thermal_conductivity","Viscous_dissipation ","Adiabatic_heating   ","Internal_heating    ","Density-normalised  ","Phase               ","Lyapunov_exponent   ","Basalt              ","Harzburgite         ","TTG                 ","Basaltic_crust      ","TTG_crust           ","Continental_crust   ","Air                 ","Primordial          ","HPE                 ","Impactor            ","Water               ","Carbon              ", "Nitrogen            ","Metal               ","FeO                 ","Age                 ","Disl_creep_fraction ","GBS_creep_fraction  ","Peierls_creep_fract.","Plasticity_fraction ","Grain_size          ","Deformation_mode    ","Heat_capacity       ","Thermal_expansivity ","Dynamic_Pressure    ","Tracers_per_cell    ","Tracer-based density","Water_solubility    ","Bulk_sound_velocity ","P-wave_velocity     ","S-wave_velocity     ","Phase_Ol_1          ","Phase_Ol_2          ","Phase_Ol_3          ","Phase_Ol_4          ","Phase_Ol_5          ","Phase_PxGt_1        ", "Phase_PxGt_2        ","Phase_PxGt_3        ","Phase_PxGt_4        ","Phase_PxGt_5        ")
fileExt2FieldName = Dict(fieldcen .=> fieldname)
fieldName2FileExt = Dict(fieldname .=> fieldcen)

function read_magic(filename;closeFile = false)

    inputILEN = 4
    byte_reverse_in = false

    # Open file in binary mode
    io = open(filename, "r")

    # Read the first 4 bytes as a 32-bit integer
    @show magic = binRead(io, Int32)

    if !valid_magic(magic) && magic > 8000  # Try 64-bit
        magic -= 8000
        if valid_magic(magic)
            binRead(io, Int32)  # Read an extra 4 bytes (dummy)
            inputILEN = 8
        end
    end

    

    if !valid_magic(magic)  # Try 32-bit with byte reversal
        byte_reverse_in = true
        close(io)
        io = open(filename, "r")
        magic = bswap(binRead(io, Int32))  # Swap bytes for endianness
    end

    if !valid_magic(magic) && magic > 8000  # Try 64-bit with byte reversal
        magic -= 8000
        byte_reverse_in = true
        if valid_magic(magic)
            binRead(io, Int32)   # Read an extra 4 bytes (dummy)
            inputILEN = 8
        end
    end

    if closeFile
        close(io)
    end
    
    return magic, inputILEN, byte_reverse_in, io
end

function binRead(io,dataTypeTmp::DataType)

    value=read(io,dataTypeTmp)
        
    return value
end


function binRead(io,dataTypeTmp::DataType,numberOfElements)

    values=[]
    for i in 1:numberOfElements
        value=binRead(io,dataTypeTmp)
        values=push!(values,value)
    end
    return values
end




# Dummy function for validation (replace with real logic)
function valid_magic(m)
    magicLogic = false
    if (m>=1 && m<=12) || (m>=301 && m<=312) || (m>=401 && m<=412) || (m>=101 && m<=112) 
        magicLogic=true
    end

    return magicLogic   # Replace with real valid values
end

function evaluate_nval_from_magicNumber(magic)
    nval=1
    if magic > 400
        nval = 4     #  v+p (new files)
    elseif magic > 300
        nval = 3     # v only (old version)
    end
    magic = mod(magic,100)
    return magic, nval
end




function read_header(io,inputILEN,magic)
    # assume integer and real have same precision in input file
    if inputILEN == 4
        intDataType=Int32
        floatDataType=Float32
    elseif inputILEN == 8
        intDataType=Int64
        floatDataType=Float64
    end
    @show inputILEN
    

    nx,ny,nz=binRead(io,intDataType,3)
    
    nb=1
    if magic>=7
        nb= binRead(io,intDataType)
    end


    asp_ratio=binRead(io,floatDataType,2) # asp_ratio[1] = asp_ratio[2]/ny 

    @show nnx,nny,nnz=binRead(io,intDataType,3) # partition numbers

    nnb=1
    if magic>=8
        nnb = binRead(io,intDataType)
    end
    
    

    if magic >=2
        tmpzc=binRead(io,floatDataType,2*(nz+1)-1)
        tmpzc=push!(tmpzc,0.0) # 0.0 is a dummy # who did this??

        zc = reshape(tmpzc,(2,nz+1)) # 2 columns because of the staggered grids, we use the first for plotting (150 km of atmosphere)
    else # normally this should not be the case
        dz = 1.0 / nz
        zc=zeros(floatDataType,2,nz+1)
        for iz in 1:nz
            zc[1,iz] = iz*dz
            zc[2,iz] = dz*(iz+0.5)
        end 
    end
    
    if magic >=7 # core radius for cylindrical or spherical geometry
        @show rcmb= binRead(io,floatDataType)

        if rcmb==-1.
            boolSpherical= false  #cartesian
        else
            boolSpherical= true  # spherical
        end 
    end 


    if boolSpherical
        dx = asp_ratio[1]/nx
        dy = asp_ratio[2]/ny
    else
        dx = asp_ratio[1]/nx * (zc[1,nz+1]-zc[1,1])
        dy = asp_ratio[2]/ny * (zc[1,nz+1]-zc[1,1])
    end 


    if magic >= 3
        @show iStep = binRead(io,intDataType)
        @show time =  binRead(io,floatDataType)
    else
        istep = 0
        time = 0.0
    end

    erupta = zeros(floatDataType,2)

    if magic >= 5 
        erupta[1] = binRead(io,floatDataType)
    end
    if magic >= 12
        erupta[2] = binRead(io,floatDataType)
        intruda = binRead(io,floatDataType,2)
        TTGmass = binRead(io,floatDataType,3)
    end
    if magic >= 6
        botT_val = binRead(io,floatDataType)
    end
    T_core=botT_val
    if magic >= 10
        T_core = binRead(io,floatDataType)
    end
    if magic >= 11
        water_budget = binRead(io,floatDataType)
    end
    if magic >= 4
        binRead(io,floatDataType,nx+ny+nz)
    end

    #usefulHeaders = intDataType,floatDataType,dx,dy,nx,ny,nz,nnx,nny,nnz,nb,rcmb,iStep,time,zc,boolSpherical

    return intDataType,floatDataType,dx,dy,nx,ny,nz,nnx,nny,nnz,nb,rcmb,iStep,time,zc,boolSpherical
end

read_header (generic function with 1 method)

In [ ]:
Interpolator=nothing
maxR = nothing
minR = nothing
maxθ = nothing
minθ = nothing
maxϕ = nothing
minϕ = nothing

boolFlat = false



for file in compositions[1:1]
   @show file
   @show magic, inputILEN, byte_reverse_in, io=read_magic(file)
   @show magic,nval = evaluate_nval_from_magicNumber(magic)
   @show intDataType,floatDataType,dx,dy,nx,ny,nz,nnx,nny,nnz,nb,rcmb,iStep,time,zc,boolSpherical=read_header(io,inputILEN,magic)

   
   nodes=nothing


   
   if nx == 1
      boolFlat = true
   end



   if boolSpherical
      # these are the grid points including the boundaries
      r = rcmb .+ zc[1,1:nz+1]
      
      theta = (collect(1:1:nx+1) .- (nx+1)) .* dx .+ 0.5*π
      phi = (collect(1:1:ny+1) .- (ny+1)) .* dy
      

      # these are the midpoints 
      r_mid = rcmb .+ zc[2,1:nz]
      theta_mid = (collect(1:1:nx) .- (nx+1)) .* dx .+ 0.5*π .+ 0.5*dx
      phi_mid = phi = (collect(1:1:ny) .- (ny+1)) .* dy .+ 0.5*dy
      
      # interpolation should be done with the midpoints + end points

      r_new = prepend!(r_mid,r[1])
      r_new = push!(r_mid,r[end])
      minR = r[1]
      maxR = r[end]

      theta_new = prepend!(theta_mid,theta[1])
      theta_new = push!(theta_new,theta[end])
      minθ = theta[1]
      maxθ = theta[end]
      phi_new = prepend!(phi_mid,phi[1])
      phi_new = push!(phi_new,phi[end])
      minϕ = phi[1]
      maxϕ = phi[end]

      if boolFlat
         minθ = 0.
         maxθ = 0.
         theta_new = (0.)  
         nodes=(phi_new,r_new)
      else   
         nodes=(theta_new,phi_new,r_new)
      end
  
   else
      # do not know if this works (certainly not! try to mimic the code above for the spherical case)
      x=collect(1:1:nx+1) .* dx
      y=collect(1:1:ny+1) .* dy
      z=zc
      nodes=(x,y,z)
   end

   


   #coord[1,ix,iy,iz] = r[iz]*sin(theta)*cos(phi)
   #coord[2,ix,iy,iz] = r[iz]*sin(theta)*sin(phi)
   #coord[3,ix,iy,iz] = r[iz]*cos(theta)
   
   
   

   field = binRead(io,Float32,(nx)*(ny)*(nz))
   
   newField = nothing

   if boolFlat
      field=reshape(field, (ny,nz)) 
      newField = zeros(floatDataType,ny+2,nz+2)

      # the interior

      newField[2:ny+1,2:nz+1] = field[1:end,1:end]

      # 4 'surfaces'

      newField[1,2:nz+1] = field[1,1:nz]
      newField[end,2:nz+1] = field[end,1:nz]
      newField[2:ny+1,1] = field[1:ny,1]
      newField[2:ny+1,end] = field[1:ny,end]

      # 4 'endpoints'

      newField[1,1,1]=field[1,1,1]
      newField[1,end,1]=field[1,end,1]
      newField[1,1,end]=field[1,1,end]
      newField[1,end,end]=field[1,end,end]


   else
      field=reshape(field, (nx,ny,nz)) 
      newField = zeros(floatDataType,nx+2,ny+2,nz+2)

      # the interior

      newField[2:nx+1,2:ny+1,2:nz+1] = field[1:nx,1:ny,1:nz]

      # 6 'surfaces'
      
      newField[1,2:ny+1,2:nz+1] = field[1,1:ny,1:nz]
      newField[end,2:ny+1,2:nz+1] = field[end,1:ny,1:nz]
      newField[2:nx+1,1,2:nz+1] = field[1:nx,1,1:nz]
      newField[2:nx+1,end,2:nz+1] = field[1:nx,end,1:nz]
      newField[2:nx+1,2:ny+1,1] = field[1:nx,1:ny,1]
      newField[2:nx+1,2:ny+1,end] = field[1:nx,1:ny,end]

      # 8 'endpoints'
      newField[1,1,1]=field[1,1,1]
      newField[end,1,1]=field[end,1,1]
      newField[1,end,1]=field[1,end,1]
      newField[end,end,1]=field[end,end,1]
      newField[1,1,end]=field[1,1,end]
      newField[end,1,end]=field[end,1,end]
      newField[1,end,end]=field[1,end,end]
      newField[end,end,end]=field[end,end,end]

   end

   fieldInterpolated=interpolate(nodes,newField,Gridded(Linear()))

   Interpolator=fieldInterpolated
end

X=collect(-6500e3:500e3:6500e3)
Y=collect(-6500e3:500e3:6500e3)
Z=collect(-6500e3:500e3:6500e3)

#X=collect(0:200e6:6500e6)
#Y=collect(0:200e6:6500e6)
#Z=collect(0:200e6:6500e6)
if boolFlat
   Z = (0.)
end


count = 0

newNx=size(X,1)
newNy=size(Y,1)
newNz=size(Z,1)
fieldCartesian=zeros(Float64,newNx,newNy,newNz)

for iz in 1:size(Z,1) 
   z=Z[iz]
   for iy in 1:size(Y,1)
      y=Y[iy]
      for ix in 1:size(X,1)
         x=X[ix]
         r = sqrt(x^2+y^2+z^2)

         theta =nothing

         if boolFlat
            theta = π/2
         else
            theta = acos(z/r)
         end

         if r == 0 
            phi =0
         else            
            #rxy = sqrt(x^2+y^2)
         

            if x == 0. && y >= 0
               phi = 0
            elseif x == 0. && y < 0
               phi = -π
            elseif x < 0 && y > 0
               phi = atan(y/x) + π
            elseif x < 0 && y < 0
               phi = atan(y/x) - π
            else
               phi = atan(y/x)
            end

            #@show minϕ, maxϕ
            #@show r, theta, phi
            if boolFlat
               if (r-minR)*(r-maxR) <= 0  && (phi-minϕ)*(phi-maxϕ) <= 0
                  count += 1

                  fieldCartesian[ix,iy,iz]= Interpolator(phi,r)
               end
            else
               if (r-minR)*(r-maxR) <= 0 && (theta-minθ)*(theta-maxθ) <= 0 && (phi-minϕ)*(phi-maxϕ) <= 0
               #@show r, theta, phi  
                  count += 1
               end
            end
         end   
         
      end
   end
end

@show count


In [ ]:
fieldCartesian

In [ ]:
Interpolator(-0.5, 4.e6)

In [ ]:
@show minθ, maxθ, minϕ, maxϕ, minR, maxR

In [ ]:
X

In [ ]:
Y=collect(0.:20e6:0.)

In [ ]:
newField(1.5646603892948965,-1.54932421875, 1.5907725225219727e6)

In [ ]:
A = rand(8,20)
nodes = ([x^2 for x = 1:8], [0.2y for y = 1:20])
itp = interpolate(nodes, A, Gridded(Linear()))
itp(4,1.2)  # approximately A[2,6]

In [ ]:
A = rand(20)
nodes = rand(20,2)
itp = interpolate(nodes, A, Gridded(Linear()))
itp(4,1.2)

In [ ]:
nodes

In [ ]:
nx=6
dx=1
x=((1:1:nx+1) .- (nx+1)) .* dx .+ 0.5*π

x=prepend!(collect(x),[4.0])

In [ ]:
nx=5
dx=1
theta = ((1:1:nx+1) .- (nx+1)) .* dx .+ 0.5*π
size(theta)

In [ ]:
newColorBar,newValues=regenerataionColorMap(colorbar,values,Ncolor)
        scene = heatmap(transpose(float_array), colormap =  newColorBar,colorrange=(values[1],values[end]))
        display(scene)